# Gemini API Key usage

An API key is a unique identifier that authenticates a user or application to an API. It is used to access protected resources or perform actions on behalf of the user or application.

We **don’t** need Google Cloud Platform (GCP) to Use Gemini. We can use Gemini directly by using the Gemini API key. To get an API key, we need to:

1. **Go to Google AI Studio** (which is separate from GCP)
    - [https://aistudio.google.com/](https://aistudio.google.com/)  

2. **Sign in with the Google Account**  
    - We can use our personal or work Google account.  
    - No need for GCP billing setup!

3. **Get an API Key**  
    - Click on **API keys**.  
    - Click **"Create API Key"**. 
    - Copy the key (it will look something like: `AIzaSyD-YourGeneratedKeyHere`).  

4. **Prepare the API Key for usage within the Code**
    - Place the API key in the `.env` file with a `GENAI_API_KEY` variable.
    - Within the `src/credentials.py` file, add the following code:
        ```python
        import os
        from dotenv import load_dotenv
        load_dotenv()
        GENAI_API_KEY = os.getenv("GENAI_API_KEY")
        ```

## **Resources:**
- [Gemini Developer API](https://ai.google.dev/gemini-api/docs?_gl=1*1eax6ht*_ga*NjM0MTcxMjEzLjE3Mzk3MTEyOTI.*_ga_P1DBVKWT6V*MTczOTcxMTI5Mi4xLjAuMTczOTcxMTI5Mi42MC4wLjE1MTE5ODE2MTY.#python)

---

## Gemini API Key Quick Usage

```bash
curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key=GEMINI_API_KEY" \
-H 'Content-Type: application/json' \
-X POST \
-d '{
  "contents": [{
    "parts":[{"text": "Explain how AI works"}]
    }]
   }'
```

Note: If we want **advanced features** like **enterprise access, private models, or deploying AI at scale**, then **Vertex AI on GCP is better**.

---


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google import genai
from genaianalysis.credentials import GENAI_API_KEY

client = genai.Client(api_key=GENAI_API_KEY)

## Hello, World!

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works",
)

print(response.text)

Okay, let's break down how AI works. It's a broad field, so I'll focus on the core concepts and then touch on specific techniques.

**The Basic Idea: Mimicking Intelligence**

At its heart, AI aims to create machines that can perform tasks that typically require human intelligence.  These tasks might include:

*   **Learning:** Acquiring information and rules for using that information.
*   **Reasoning:**  Drawing conclusions from information.
*   **Problem-solving:** Finding solutions to complex issues.
*   **Perception:** Understanding sensory input (like images, sound, or text).
*   **Natural Language Understanding:**  Comprehending and responding to human language.

**Key Components and Concepts:**

1.  **Data:**  AI systems *learn* from data. The more high-quality data an AI system has, the better it can typically perform. Data can be anything from images of cats and dogs (for image recognition) to text of customer reviews (for sentiment analysis) to records of transactions (for f

## Generate structured output

Here are a few use cases that might require structured output from the model:

- Build a database of companies by pulling company information out of newspaper articles.
- Pull standardized information out of resumes.
- Extract ingredients from recipes and display a link to a grocery website for each ingredient.

To extract structured output from the model, we can:

- Supply a schema as text in the prompt
- Supply a schema through model configuration

### Supply a schema as text in the prompt

In [ ]:
prompt = """List a few of the most popular restaurants in Medellin, Colombia in JSON format.

Use this JSON schema:

restaurants = {'neighborhood': str, 'restaurants': list[str]}
Return: list[restaurants]"""

client = genai.Client(api_key=GENAI_API_KEY)
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt,
)

# Use the response as a JSON string.
print(response.text)

```json
[
  {
    "neighborhood": "El Poblado",
    "restaurants": [
      "El Cielo",
      "Carmen",
      "Oci.mde",
      "In Situ"
    ]
  },
  {
    "neighborhood": "Laureles",
    "restaurants": [
      "Alambique",
      "Matiz",
      "Galápagos Cocina Peruana"
    ]
  },
    {
    "neighborhood": "Envigado",
    "restaurants": [
      "Zumo"
    ]
  }
]
```


### Supply a schema through model configuration

In [5]:
from pydantic import BaseModel, TypeAdapter

class Restaurants(BaseModel):
  neighborhood: str
  restaurants: list[str]
  
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few of the most popular restaurants in Medellin, Colombia.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Restaurants],
    },
)
# Use the response as a JSON string.
print(response.text)

[
  {
    "neighborhood": "El Poblado",
    "restaurants": ["Carmen", "El Cielo", "In Situ"]
  },
  {
    "neighborhood": "Laureles",
    "restaurants": ["Alambique", "Matriarca"]
  },
  {
    "neighborhood": "Envigado",
    "restaurants": ["Ammazza Gin Garden"]
  }
]


In [12]:
import json

# Use instantiated objects.
restaurants_list: list[Restaurants] = response.parsed
display(restaurants_list)

display(json.loads(response.text))


[Restaurants(neighborhood='El Poblado', restaurants=['Carmen', 'El Cielo', 'In Situ']),
 Restaurants(neighborhood='Laureles', restaurants=['Alambique', 'Matriarca']),
 Restaurants(neighborhood='Envigado', restaurants=['Ammazza Gin Garden'])]

[{'neighborhood': 'El Poblado',
  'restaurants': ['Carmen', 'El Cielo', 'In Situ']},
 {'neighborhood': 'Laureles', 'restaurants': ['Alambique', 'Matriarca']},
 {'neighborhood': 'Envigado', 'restaurants': ['Ammazza Gin Garden']}]